In [ ]:
# 필요한 라이브러리 import
import cv2
import ipywidgets.widgets as widgets
import threading
import time

# 카메라 이미지 위젯 생성 (JPEG 형식, 640x480 해상도)
image_widget = widgets.Image(format="jpeg", width=640, height=480)

In [ ]:
# ============================================
# 이미지 변환 관련 함수
# ============================================


def convert_bgr8_to_jpeg(bgr_image, quality=75):
    """
    BGR8 형식의 이미지를 JPEG 형식으로 변환합니다.

    Args:
        bgr_image: BGR8 형식의 OpenCV 이미지 (numpy array)
        quality: JPEG 품질 (1-100, 기본값: 75)

    Returns:
        bytes: JPEG 형식의 바이트 데이터
    """
    if bgr_image is None:
        return None

    try:
        # JPEG 인코딩 파라미터 설정
        encode_params = [cv2.IMWRITE_JPEG_QUALITY, quality]
        # BGR 이미지를 JPEG로 인코딩
        success, encoded_image = cv2.imencode(".jpg", bgr_image, encode_params)

        if not success:
            print("JPEG encoding failed.")
            return None

        return bytes(encoded_image)
    except Exception as e:
        print(f"Error converting image: {e}")
        return None

In [ ]:
# ============================================
# 카메라 초기화 및 설정 관련 함수
# ============================================


def init_camera(camera_index=0, width=640, height=480):
    """
    카메라를 초기화하고 기본 설정을 적용합니다.

    Args:
        camera_index: 카메라 장치 인덱스 (기본값: 0, /dev/video0)
        width: 이미지 너비 (기본값: 640)
        height: 이미지 높이 (기본값: 480)

    Returns:
        cv2.VideoCapture: 초기화된 카메라 객체, 실패 시 None
    """
    try:
        # 카메라 열기
        camera = cv2.VideoCapture(camera_index)

        if not camera.isOpened():
            print(f"Error: Cannot open camera {camera_index}.")
            return None

        # 해상도 설정
        camera.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        camera.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

        # 추가 설정 (주석 처리된 옵션들)
        # camera.set(cv2.CAP_PROP_FPS, 30)  # 프레임 레이트 설정
        # camera.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
        # camera.set(cv2.CAP_PROP_BRIGHTNESS, 40)  # 밝기 설정 (-64 ~ 64, 기본값: 0.0)
        # camera.set(cv2.CAP_PROP_CONTRAST, 50)    # 대비 설정 (-64 ~ 64, 기본값: 2.0)
        # camera.set(cv2.CAP_PROP_EXPOSURE, 156)   # 노출 설정 (1.0 ~ 5000, 기본값: 156.0)

        return camera
    except Exception as e:
        print(f"Error initializing camera: {e}")
        return None


def read_camera_frame(camera):
    """
    카메라로부터 프레임을 읽어옵니다.

    Args:
        camera: cv2.VideoCapture 객체

    Returns:
        tuple: (성공 여부, 프레임 이미지), 실패 시 (False, None)
    """
    if camera is None:
        return (False, None)

    try:
        success, frame = camera.read()
        return (success, frame)
    except Exception as e:
        print(f"Error reading frame: {e}")
        return (False, None)


def release_camera(camera):
    """
    카메라 리소스를 해제합니다.

    Args:
        camera: cv2.VideoCapture 객체
    """
    if camera is None:
        return

    try:
        camera.release()
        print("Camera released.")
    except Exception as e:
        print(f"Error releasing camera: {e}")


# ============================================
# 카메라 초기화 및 테스트
# ============================================
# 1. 카메라 초기화
image = init_camera(camera_index=0, width=640, height=480)

# 2. 초기 프레임 읽기 및 위젯에 표시
if image is not None:
    success, frame = read_camera_frame(image)
    if success:
        jpeg_data = convert_bgr8_to_jpeg(frame)
        if jpeg_data is not None:
            image_widget.value = jpeg_data
    else:
        print("Failed to read initial frame.")
else:
    print("Failed to initialize camera.")

In [ ]:
# ============================================
# 카메라 스트리밍 메인 함수
# ============================================


def main_camera_streaming(camera, image_widget, frame_interval=0.010):
    """
    카메라 스트리밍 메인 루프

    Args:
        camera: cv2.VideoCapture 객체
        image_widget: ipywidgets.Image 위젯
        frame_interval: 프레임 간 대기 시간 (초 단위, 기본값: 0.010초)
    """
    if camera is None:
        print("Error: Camera is not initialized.")
        return

    try:
        # 1. 이미지 위젯 표시
        display(image_widget)

        # 2. 메인 스트리밍 루프
        while True:
            # 2-1. 프레임 읽기
            success, frame = read_camera_frame(camera)

            if not success:
                print("Failed to read frame.")
                break

            # 2-2. BGR8을 JPEG로 변환
            jpeg_data = convert_bgr8_to_jpeg(frame)

            if jpeg_data is None:
                print("Failed to convert image.")
                continue

            # 2-3. 위젯에 이미지 업데이트
            image_widget.value = jpeg_data

            # 2-4. 프레임 간격 대기
            time.sleep(frame_interval)

    except KeyboardInterrupt:
        # 3. 사용자 중단 시 종료
        print("\nProgram closed.")
    except Exception as e:
        print(f"Streaming error: {e}")


# ============================================
# 메인 실행
# ============================================
if image is not None:
    main_camera_streaming(image, image_widget, frame_interval=0.010)
else:
    print("Cannot start streaming: camera is not initialized.")

In [ ]:
# ============================================
# 프로그램 종료 시 정리 작업
# ============================================
# 카메라 객체를 해제하여 다음 프로그램에서 사용할 수 있도록 함
# (해제하지 않으면 카메라 모듈이 점유되어 다른 프로그램에서 사용 불가)
release_camera(image)